### MNIST Neural Network 'from scratch'
<br\>
This notebook implements 'from scratch' a Neural Network for recognizing hand written digits, using the MNIST database. By 'from scratch' we mean without using any library like `tensorflow` or `pytorch`. The gradients are calculated by implementing the backpropagation algorithm explicitely, without resorting to automatic differentiation techniques.

In [1]:
import pickle
import gzip
import numpy as np
import timeit

In [2]:
# Global functions used by the neural network.

def sigmoid(z):
    """The sigmoid function."""
    return 1.0 / (1.0 + np.exp(-z))

def D_sigmoid(z):
    """The derivative of the sigmoid function."""
    return sigmoid(z) * (1 - sigmoid(z))

def D_cost(output, target):
    """The derivative of the quadratic cost function."""
    return (output - target)

In [3]:
class NeuralNetworkMNIST(object):
    """
    A Neural Network designed to be trained on the MNIST database. 
    """
    
    def __init__(self, layersz):
        """
        Initializes biases and weights with random values from a N(0,1) distribution.
        Since we are using this network with the MNIST database, the input layer must
        be of size 784 = 28 x 28 (the number of pixels of each image). Also, the ouput
        layer must be of size 10 (to represent 0...9)

        Parameters:
        layersz -- list of layer sizes, layers[0] corresponding to the input layer
        """
        if layersz [0] != 784: raise RuntimeError('The size of the input layer must be 784')
        if layersz[-1] !=  10: raise RuntimeError('The size of the output layer must be 10')
        
        # the following convention is used for weights: 
        # w[i,j] denotes the weight associated with the connection from neuron
        # 'j' in the previous layer to the neuron 'i' in the current layer.
        
        self.nlayers = len(layersz)
        self.layersz = layersz
        self.biases  = [np.random.randn(i, 1) for i in layersz[1:]]
        self.weights = [np.random.randn(j, i) for i, j in zip(layersz[:-1], layersz[1:])]

        
    def feedforward(self, a):
        """
        Propagates a given input vector forward through the network and returns the output.
        
        Parameters:
        a -- network input, a 2D array of shape (n,1), where n is the size of the input layer        
        """
        if a.shape != (self.layersz[0], 1):
            raise RuntimeError('Input array has wrong shape')
        
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    
    def backpropagate(self, x, y):
        """
        Calculates the gradient of the cost function for a given  
        input vector using the backpropagation algorithm.
        
        Parameters:
        x -- network input,  a 2D array of shape (n,1), where n is the size of the input  layer
        y -- target  output, a 2D array of shape (n,1), where n is the size of the output layer
        
        Returns:
        A tuple consisting of two lists.
        The first  one stores the derivative of the cost function with respect to the bias.
        The second one stores the derivative of the cost function with respect to the weights.
        In each list, the entries are arrays, and correspond to the network layers.
        For example, the first entry in the first list is the 1D array of derivatives of the
        cost function with respect to the biases for the neurons in the input layer.
        Similarly, the first entry in the second list is the 2D array of derivatives of the cost
        function with respect to the weights associated with the connections between the neurons
        in the input and first hidden layer.
        """

        if x.shape != (self.layersz[0] , 1): raise RuntimeError( 'Input array has wrong shape')
        if y.shape != (self.layersz[-1], 1): raise RuntimeError('Output array has wrong shape')
           
        # create arrays to store the gradient of the cost function 
        dC_dBias   = [np.zeros(b.shape) for b in self.biases ]   # <-- derivs of C with respect to biases
        dC_dWeight = [np.zeros(w.shape) for w in self.weights]   # <-- derivs of C with respect to weights
        
        # -- FORWARD PASS
        
        activation  =  x
        activations = [x]
        zs          = [ ]    
        
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, activation) + b
            zs.append(z)            
            activation = sigmoid(z)
            activations.append(activation)
        
        # -- BACKWARD PASS
        
        # output layer
        delta          = D_cost(activations[-1], y) * D_sigmoid(zs[-1])  # <-- deriv of 'C' with respect to 'z'
        dC_dBias  [-1] = delta
        dC_dWeight[-1] = np.dot(delta, activations[-2].T)
               
        # backpropagate
        for L in range(2, self.nlayers):           
            delta          = np.dot(self.weights[-L+1].T, delta) * D_sigmoid(zs[-L])
            dC_dBias  [-L] = delta
            dC_dWeight[-L] = np.dot(delta, activations[-L-1].T)
            
        return (dC_dBias, dC_dWeight)                               
                                
        
    def SGD(self, training_data, epochs, batchsz, eta, test_data=None):
        """
        Train the neural network using batch stochastic gradient descent.  
        The network weights and biases are updated as the result of running this method.
        Both 'training_data' and 'test_data' are lists of tuples, each tuple being an
        example - the first element is the network input, the second is the target output.
        
        Parameters:
        training_data -- list of tuples representing training inputs and the desired outputs  
        epochs        -- for how many epochs to train the network
        batchsz       -- the size of each batch of training example (this is *stochastic* GD)
        eta           -- the learning rate
        test_data     -- used to evaluate the performace of the network at the end of each epoch
        """
        for j in range(epochs):
            start_time = timeit.default_timer()
                
            # break up the training data into batches
            np.random.shuffle(training_data)
            batches = [training_data[k:k+batchsz] for k in range(0, len(training_data), batchsz)]
            
            # SGD means that we update weights/biases based on gradients calculated
            # using only a batch of training examples (as opposed to the entire training data) 
            for batch in batches:
                dC_dBias_sum   = [np.zeros(b.shape) for b in self.biases ]
                dC_dWeight_sum = [np.zeros(w.shape) for w in self.weights]

                # calculate the (stochastic) gradient
                # below 'x' represents an input vector, 'y' the target output
                for x, y in batch:
                    dC_dBias, dC_dWeight = self.backpropagate(x, y)
                    dC_dBias_sum   = [bs+b for bs, b in zip(dC_dBias_sum  , dC_dBias)  ]
                    dC_dWeight_sum = [ws+w for ws, w in zip(dC_dWeight_sum, dC_dWeight)]

                # update weights/biases in the direction of the stochastic gradient
                eta_avg = eta/len(batch)
                self.weights = [w - eta_avg * deriv_w for w, deriv_w in zip(self.weights, dC_dWeight_sum)]
                self.biases  = [b - eta_avg * deriv_b for b, deriv_b in zip(self.biases , dC_dBias_sum)  ]
            
            dt = timeit.default_timer() - start_time
            if test_data: print("Epoch %2d: %d of %d (elapsed time: %fs)" % (j+1, self.evaluate(test_data), len(test_data), dt))
            else:         print("Epoch %2d complete  (elapsed time: %fs)" % (j+1), dt)

                        
    def evaluate(self, test_data):
        """
        Evaluates the performance of the neural network on a data set not used for training.
        """
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

In [4]:
# where to find the file storing the MNIST database
MNIST_DATA_FILEPATH = "mnist.pkl.gz"

def load_data_raw():
    """
    Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The 'training_data' is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the 'training_data' tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The 'validation_data' and 'test_data' are similar, except
    each contains only 10,000 images.
    """
    f = gzip.open(MNIST_DATA_FILEPATH, 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)


def load_data():
    """
    Repackages the data returned by 'load_data_raw' in a format
    more convenient for using with the neural network.
    
    Return a tuple (training_data, validation_data, test_data).

    'training_data'   is a list of 50,000 2-tuples (x, y)
    'validation_data' is a list of 10,000 2-tuples (x, z)
    'test_data'       is a list of 10,000 2-tuples (x, z)

    'x' is a numpy array of shape (784, 1) containing the input image.
    'y' is a numpy array of shape ( 10, 1) representing the digit encoded
        by 'x' (it has 0 entries with the exception of one 1 in the position
        of the digit represented by 'x')
    'z' is just the digit represented by 'x'
    """
    tr_d, va_d, te_d = load_data_raw()

    training_inputs   = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results  = [asvector(y) for y in tr_d[1]]
    training_data     = zip(training_inputs, training_results)

    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data   = zip(validation_inputs, va_d[1])
    
    test_inputs       = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data         = zip(test_inputs, te_d[1])

    return (training_data, validation_data, test_data)


def asvector(j):
    """Create vector of shape (10, 1) with 1.0 in the jth position and 0.0 elsewhere."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

# load the MNIST databse
print("Loading the MNIST database...")
training_data, validation_data, test_data = [list(d) for d in load_data()]
print("Done")

Loading the MNIST database...
Done


In [5]:
# initialize a Neural Network

layer_sizes = [784, 30, 10]  # <-- first entry must be 784, last one must be 10

net = NeuralNetworkMNIST(layer_sizes)

In [6]:
# evaluate the performance of the untrained network

nright = net.evaluate(test_data)
print("Untrained network: got right %d out of %d (accuracy %.2f pct)" % (nright, len(test_data), 100*float(nright)/len(test_data)))

Untrained network: got right 893 out of 10000 (accuracy 8.93 pct)


In [7]:
# train the network

EPOCHS  = 10
BATCHSZ = 10
ETA     =  2

net.SGD(training_data, EPOCHS, BATCHSZ, ETA, test_data=test_data)

Epoch  1: 8192 of 10000 (elapsed time: 5.643085s)
Epoch  2: 8303 of 10000 (elapsed time: 5.700366s)
Epoch  3: 9247 of 10000 (elapsed time: 5.626254s)
Epoch  4: 9310 of 10000 (elapsed time: 5.682167s)
Epoch  5: 9311 of 10000 (elapsed time: 5.808394s)
Epoch  6: 9332 of 10000 (elapsed time: 5.701996s)
Epoch  7: 9355 of 10000 (elapsed time: 5.638000s)
Epoch  8: 9406 of 10000 (elapsed time: 5.720068s)
Epoch  9: 9392 of 10000 (elapsed time: 5.707201s)
Epoch 10: 9442 of 10000 (elapsed time: 5.674460s)
